In [6]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import re
import math
import random
import jieba
import numpy as np
from six.moves import xrange
import tensorflow as tf

In [7]:
#读取数据
def read_data():
    """
    对要训练的文本进行处理，最后把文本的内容的所有词放在一个列表中
    """
    #读取停用词
    stop_words = []
    with open('data/stop_words.txt',"r",encoding="UTF-8") as f:
        line = f.readline()
        while line:
            stop_words.append(line[:-1])
            line = f.readline()
    stop_words = set(stop_words)
    print('停用词读取完毕，共{n}个词'.format(n=len(stop_words)))

    # 读取文本，预处理，分词，得到词典
    raw_word_list = []
    rules =  u"([\u4e00-\u9fff]+)"
    pattern =  re.compile(rules)
    with open('data/笑傲江湖.txt',"r", encoding='UTF-8') as f:
        lines = f.readlines()
        for line in lines:
            line = line.replace("\r","").replace("\n","").strip()
            if line == "" or line is None:
                continue
            line = ' '.join(jieba.cut(line))
            seg_list = pattern.findall(line)
            words_list = []
            for word in seg_list:
                if word not in stop_words:
                    words_list.append(word)
            with open('data/分词后的笑傲江湖.txt', 'a', encoding='utf-8') as ff:
                if len(words_list) > 0:
                    line = ' '.join(words_list) + "\n"
                    ff.write(line) # 词汇用空格分开
                    ff.flush()
                    raw_word_list.extend(words_list)
    return raw_word_list

In [8]:
#step 1:读取文件中的内容组成一个列表
words = read_data()

停用词读取完毕，共1893个词


In [19]:
print(len(words))

497355


In [20]:
print(len(set(words)))

42143


In [21]:
vocabulary_size = len(set(words))
def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    print("count",len(count))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

In [22]:
data, count, dictionary, reverse_dictionary = build_dataset(words)
del words

count 42143


In [23]:
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 1], ('的', 13183), ('了', 12312), ('道', 8508), ('是', 7650)]
Sample data [2513, 19899, 36768, 32, 38996, 20, 32635, 8465, 5184, 18722] ['笑傲江湖', '作者', '金庸', '一', '灭门', '和', '风熏', '柳', '花香', '醉人']


In [24]:
data_index = 0
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1  # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]],'->', labels[i, 0], reverse_dictionary[labels[i, 0]])

19899 作者 -> 36768 金庸
19899 作者 -> 2513 笑傲江湖
36768 金庸 -> 32 一
36768 金庸 -> 19899 作者
32 一 -> 36768 金庸
32 一 -> 38996 灭门
38996 灭门 -> 20 和
38996 灭门 -> 32 一


In [10]:
# Step 4: Build and train a skip-gram model.
batch_size = 128
embedding_size = 300  
skip_window = 1       
num_skips = 2         
valid_size = 4      #切记这个数字要和len(valid_word)对应，要不然会报错哦   
valid_window = 100  
num_sampled = 64    # Number of negative examples to sample.
#验证集
valid_word = ['令狐冲','左冷禅','林平之','桃根仙']
valid_examples =[dictionary[li] for li in valid_word]
graph = tf.Graph()
with graph.as_default():
    # Input data.
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

    # Ops and variables pinned to the CPU because of missing GPU implementation
    with tf.device('/gpu:0'):
        # Look up embeddings for inputs.
        embeddings = tf.Variable(
            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        # Construct the variables for the NCE loss
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]),dtype=tf.float32)

    # Compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each
    # time we evaluate the loss.
    loss = tf.reduce_mean(
            tf.nn.nce_loss(weights=nce_weights,biases=nce_biases, inputs=embed, labels=train_labels,
                 num_sampled=num_sampled, num_classes=vocabulary_size))

    # Construct the SGD optimizer using a learning rate of 1.0.
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # Compute the cosine similarity between minibatch examples and all embeddings.
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

    # Add variable initializer.
    init = tf.global_variables_initializer()

In [13]:
# Step 5: Begin training.
num_steps = 2000000
with tf.Session(graph=graph) as session:
    # We must initialize all variables before we use them.
    init.run()
    print("Initialized")

    average_loss = 0
    for step in xrange(num_steps):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

        # We perform one update step by evaluating the optimizer op (including it
        # in the list of returned values for session.run()
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val

        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # The average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0

        # Note that this is expensive (~20% slowdown if computed every 500 steps)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[:top_k]
                log_str = "Nearest to %s:" % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  281.51458740234375
Nearest to 令狐冲: 令狐冲, 情景, 齐叫, 够瞧的, 细加, 切不可, 正宗, 澎湃,
Nearest to 左冷禅: 左冷禅, 玄理, 梳子, 圆球, 催劲, 品尝, 余一片, 真挚,
Nearest to 林平之: 林平之, 远眺, 勃勃, 年糕, 洞壁, 秦, 动机, 奸人,
Nearest to 桃根仙: 桃根仙, 名正言顺, 遭遇, 终, 不忧, 乱划, 山报, 没吞油入,
Average loss at step  2000 :  106.21931360054016
Average loss at step  4000 :  49.19005161976814
Average loss at step  6000 :  30.781496989250183
Average loss at step  8000 :  21.514052735328676
Average loss at step  10000 :  16.18191594231129
Nearest to 令狐冲: 令狐冲, 他, 情景, 镗, 齐叫, 阿琳, ，, 仪琳,
Nearest to 左冷禅: 左冷禅, 圆球, 梳子, 决不会, 岳夫人, 怒火, 品尝, 林平,
Nearest to 林平之: 林平之, 远眺, 秦, 祖千秋, 奸人, 勃勃, 不易, 一条,
Nearest to 桃根仙: 桃根仙, 终, 遭遇, 纯熟, 被囚, 极强, 白袍, 没吞油入,
Average loss at step  12000 :  10.289122104883194
Average loss at step  14000 :  8.850086517095566
Average loss at step  16000 :  7.692424201011658
Average loss at step  18000 :  6.949875958561897
Average loss at step  20000 :  6.295983515501022
Nearest to 令狐冲: 令狐冲, 他, 你, 仪琳, 我, 镗, 情景, 阿琳,
Nearest to

In [14]:
# Step 6: Visualize the embeddings.
def plot_with_labels(low_dim_embs, labels, filename='tsne.png',fonts=None):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18))  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(label,
                    fontproperties=fonts,
                    xy=(x, y),
                    xytext=(5, 2),
                    textcoords='offset points',
                    ha='right',
                    va='bottom')
    plt.savefig(filename,dpi=600)
try:
    from sklearn.manifold import TSNE
    import matplotlib.pyplot as plt
    from matplotlib.font_manager import FontProperties
    
    #为了在图片上能显示出中文
    font = FontProperties(fname=r"c:\windows\fonts\simsun.ttc", size=14)
    
    tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
    plot_only = 500
    low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
    labels = [reverse_dictionary[i] for i in xrange(plot_only)]
    plot_with_labels(low_dim_embs, labels,fonts=font)
    
except ImportError:
    print("Please install sklearn, matplotlib, and scipy to visualize embeddings.")